In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog

# Função para carregar a imagem e detectar bordas
def carregar_e_processar_imagem(caminho_imagem, num_cores=20):
    # Carregar a imagem
    imagem = cv2.imread(caminho_imagem)

    # Redimensionar para facilitar o processamento (opcional)
    imagem = cv2.resize(imagem, (400, 600), interpolation=cv2.INTER_AREA)

    # Converter a imagem para escala de cinza
    img_gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

    # Aplicar detecção de bordas (Canny)
    bordas = cv2.Canny(img_gray, 100, 200)

    # Reduzir as cores da imagem para as principais (quantização)
    Z = imagem.reshape((-1, 3))  # Redimensionar para uma matriz 2D de pixels
    Z = np.float32(Z)  # Converter para float32

    # Critério de parada para o algoritmo k-means
    criterio = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)

    # Aplicar o k-means para encontrar as principais cores
    _, rótulos, centros = cv2.kmeans(Z, num_cores, None, criterio, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Converter os centros para inteiros e substituir as cores nos pixels
    centros = np.uint8(centros)
    imagem_quantizada = centros[rótulos.flatten()]
    imagem_quantizada = imagem_quantizada.reshape(imagem.shape)

    # Inverter as bordas (para fazer as linhas brancas)
    bordas_invertidas = cv2.bitwise_not(bordas)

    # Converter para 3 canais (RGB) para combinar com a imagem colorida
    bordas_rgb = cv2.cvtColor(bordas_invertidas, cv2.COLOR_GRAY2BGR)

    # Combinar as bordas com a imagem quantizada (borda branca sobre as cores)
    imagem_final = cv2.bitwise_and(imagem_quantizada, bordas_rgb)

    return imagem_final

def open_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Selecione um arquivo")
    if file_path:
        print(f"Arquivo selecionado: {file_path}")
    else:
        print("Nenhum arquivo foi selecionado.")
    return file_path

# Função para exibir a imagem
def mostrar_imagem(imagem, titulo='Imagem'):
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB))
    plt.title(titulo)
    plt.axis('off')
    plt.show()

# Caminho para sua imagem
caminho_imagem = open_file()

# Processar a imagem e gerar a imagem final
imagem_final = carregar_e_processar_imagem(caminho_imagem)

# Mostrar a imagem com as principais cores e contorno
mostrar_imagem(imagem_final, titulo='Contorno das Principais Cores')